In [ ]:
!python3 -m pip install --quiet degirum
from google.colab.patches import cv2_imshow
import numpy as np

In [ ]:
import degirum as dg

zoo = dg.connect(dg.CLOUD, 'https://cs.degirum.com/degirum/public', '<your token>')
model = zoo.load_model('yolo_v5s_coco--512x512_quant_n2x_orca1_1')
res = model('https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/ThreePersons.jpg')
cv2_imshow(res.image_overlay)

In [ ]:
# list all AI models available for inference
zoo.list_models()

# Example of how to pipeline two models
**Face Mask Detection Pipelined**


---

In [ ]:
# load models for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
face_det_model = zoo.load_model("yolo_v5s_face_det--512x512_quant_n2x_orca1_1")
mask_det_model = zoo.load_model("mobilenet_v2_mask_yn_cls--224x224_float_n2x_orca1_1")

# adjust some model properties
face_det_model.output_confidence_threshold = 0.7
face_det_model.overlay_font_scale = 1.5
face_det_model.overlay_alpha = 1
mask_det_model.overlay_show_labels = True
mask_det_model.overlay_font_scale = 1.5
mask_det_model.overlay_alpha = 1
mask_det_model.output_top_k = 1

In [ ]:
# detect faces
faces = face_det_model('https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/Mask1.jpg')

# for each detected face detect mask
with mask_det_model: # performance optimization to keep connection to mask_det_model open
    for face in faces.results:
        face_coords=np.int_(face['bbox'])
        face_box = faces.image[face_coords[1]:face_coords[3],face_coords[0]:face_coords[2]]
        mask = mask_det_model(face_box)
        face["label"] = mask.results[0]["label"]
        cv2_imshow(mask.image_overlay)

# display combined results
cv2_imshow(faces.image_overlay)

**License Plate Detection & Recognition Pipelined**


---




In [ ]:
# load models for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
lp_det_model = zoo.load_model("yolo_v5s_lp_det--512x512_quant_n2x_orca1_1")
lp_ocr_model = zoo.load_model("yolo_v5s_lp_ocr--256x256_quant_n2x_orca1_1")

# adjust some model properties
lp_det_model.output_confidence_threshold = 0.7
lp_det_model.overlay_font_scale = 1.5
lp_det_model.overlay_alpha = 1
lp_ocr_model.overlay_show_labels = True
lp_ocr_model.overlay_font_scale = 1.5
lp_ocr_model.overlay_alpha = 1
lp_ocr_model.output_top_k = 1

In [ ]:
# detect license plates
license_plates = lp_det_model('https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/Car.bmp')

# for each detected license place detect  characters
with lp_ocr_model: # performance optimization to keep connection to lp_det_model open
    for plate in license_plates.results:
        lp_coords=np.int_(plate['bbox'])
        lp_box = license_plates.image[lp_coords[1]:lp_coords[3],lp_coords[0]:lp_coords[2]]
        ocr_plates = lp_ocr_model(lp_box)
        plate["label"] = ocr_plates.results[0]["label"]
        cv2_imshow(ocr_plates.image_overlay)

# display combined results
cv2_imshow(license_plates.image_overlay)